# 📘 Proyecto: Análisis de comportamiento lector durante la pandemia
___

## 🧩 Contexto
Durante la pandemia de COVID-19, el hábito de lectura se intensificó. Muchas personas comenzaron a consumir libros desde casa, lo que impulsó el desarrollo de aplicaciones orientadas a lectores. Se nos ha proporcionado una base de datos de uno de estos servicios, con información sobre libros, autores, editoriales, calificaciones y reseñas. 
____
## 🎯Objetivos del estudio
- Analizar la estructura de la base de datos.
- Responder cinco preguntas clave mediante consultas SQL.
- Documentar hallazgos con claridad y trazabilidad.
____
# 🗃️ Exploración inicial: conexión y estructura

In [56]:
# Creamos engine para conexion a sql
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'practicum_student',       # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

____________________

## 🔍 Tablas disponibles

In [57]:
# Verificamos la conexión y vemos las tablas
query = """
select table_name
from information_schema.tables
where table_schema='public'
"""

df = pd.io.sql.read_sql(query, con =engine)
df

,table_name
0,advertisment_costs
1,authors
2,books
3,check_avg
4,orders
5,publishers
6,ratings
7,reviews
8,visits


_____
# 📚1. Total de libros publicados despues de Enero 1, 2000

In [58]:
# Obtenemos numero de libros publicados despues de Enero 1, 2000
total_libros = """
select count(*) as total
from books
where publication_date is not null and publication_date > '2000-01-01'
"""
# Leemos el resultado en un dataframe
df_total_libros = pd.io.sql.read_sql(total_libros, con=engine)

# Extraemos el numero del dataframe
total_libros = df_total_libros['total'][0]
total_libros = int(total_libros) # Convertimos a entero

# Mostramos el resultado
print(f'Se publicaron {total_libros} libros despues del 1 de Enero del 2000')


Se publicaron 819 libros despues del 1 de Enero del 2000


## 🧫Resultados
Se identificaron **819 libros** publicados después del 1 de enero del 2000, lo que demuestra una base sólida de títulos recientes que pueden ser relevantes para lectores contemporáneos.

______
## ✅2. Total de reseñas hechas por usuarios

In [60]:
# obtenemos total de reseñas hechas por usuarios y calificacion promedio para cada libro
total_reseñas = """
select count(*) as total_reseñas
from reviews
"""

# Leemos el resultado en un dataframe
df_total_reseñas = pd.io.sql.read_sql(total_reseñas, con=engine)

# Extraemos el numero del dataframe
total_reseñas = df_total_reseñas['total_reseñas'][0]

print(f'Existen {total_reseñas} reseñas de usuarios en total')

Existen 2793 reseñas de usuarios en total


## 🧫Resultados
La base contiene **2793 reseñas de texto**. Al calcular la calificación promedio por libro, se identificaron varios títulos con puntuaciones perfectas, lo que permite destacar obras altamente valoradas por la comunidad lectora.

____
## 📗3. Calificacion promedio para cada libro
Consulta que calcula la calificación promedio de cada libro, uniendo las tablas `ratings` y `books`. Se agrupa por `book_id` y `title` para asegurar unicidad y trazabilidad. El resultado se ordena de mayor a menor para destacar los libros mejor valorados.

In [61]:
# Obtenemos calificacion promedio para cada libro
average_rating = """
select b.book_id, b.title, avg(r.rating) as calificacion_promedio
from ratings as r
join books as b on r.book_id = b.book_id
group by b.book_id, b.title
order by calificacion_promedio desc
"""

# Leemos el resultado en un dataframe
df_average_rating = pd.io.sql.read_sql(average_rating, con=engine)

# Mostramos las primeras filas del dataframe
df_average_rating.head()

,book_id,title,calificacion_promedio
0,518,Pop Goes the Weasel (Alex Cross #5),5.0
1,732,The Ghost Map: The Story of London's Most Terr...,5.0
2,347,In the Hand of the Goddess (Song of the Liones...,5.0
3,610,Tai-Pan (Asian Saga #2),5.0
4,330,How to Be a Domestic Goddess: Baking and the A...,5.0


## 🧫Resultados
Se calcularon las calificaciones promedio de todos los libros registrados. Los títulos con puntuación perfecta (5.0) destacan por su alta valoración entre los lectores, lo que los convierte en candidatos ideales para recomendaciones dentro de la aplicación. 

Esto permite identificar contenido de alta calidad percibida.

____
## 🏛️4. Editorial con mayor numero de libros publicados(+50 paginas)

In [62]:
editoriales = """
select p.publisher, count(b.book_id) as total_libros
from books as b
join publishers as p on b.publisher_id = p.publisher_id
where b.num_pages > 50
group by p.publisher
order by total_libros desc
"""
# Leemos el resultado en un dataframe
df_editoriales = pd.io.sql.read_sql(editoriales, con=engine)

# Mostramos las primeras filas del dataframe
df_editoriales.head()

# Mostramos respues
print(f'La editorial con mayor numero de libros publicados con mas de 50 paginas\n es:\n\t{df_editoriales.iloc[0,0]} \ncon:\n\t{df_editoriales.iloc[0,1]} libros publicados.')


La editorial con mayor numero de libros publicados con mas de 50 paginas
 es:
	Penguin Books 
con:
	42 libros publicados.


## 🧫Resultados
Penguin Books lidera con 42 libros de más de 50 páginas, posicionándose como la editorial más activa en publicaciones sustanciales dentro del conjunto de datos.

_____
## 👤5. Autor con mayor calificacion promedio del libro(minimo 50 calificaciones)

In [63]:
# libros con mayor calificacion promedio del libro(minimo 50 calificaciones)
autores = """
select a.author, b.title, avg(r.rating) as calificacion_promedio, count(r.rating) as total_calificaciones
from authors as a
join books as b on a.author_id = b.author_id
join ratings as r on b.book_id = r.book_id
group by b.book_id, b.title, a.author
having count(r.rating) >= 50
order by calificacion_promedio desc
"""

# Leemos el resultado en un dataframe
df_autores = pd.io.sql.read_sql(autores, con=engine)

# Mostramos las primeras filas del dataframe
df_autores.head()

# Mostramos respuesta
print(f'El autor con mayor calificacion promedio del libro es:\n\t{df_autores.iloc[0,0]} \ncon su libro:\n\t"{df_autores.iloc[0,1]}" \nque tiene una calificacion promedio de:\n\t{df_autores.iloc[0,2]:.2f} \nbasada en:\n\t{df_autores.iloc[0,3]} calificaciones.')

El autor con mayor calificacion promedio del libro es:
	J.K. Rowling/Mary GrandPré 
con su libro:
	"Harry Potter and the Prisoner of Azkaban (Harry Potter  #3)" 
que tiene una calificacion promedio de:
	4.41 
basada en:
	82 calificaciones.


## 🧫Resultados
El autor con el libro mejor valorado fue **J.K. Rowling/Mary GrandPré**, con *Harry Potter and the Prisoner of Azkaban*, que obtuvo una calificación promedio de 4.41 basada en **82 evaluaciones**. Este resultado no solo destaca la popularidad sostenida de la saga de Harry Potter, sino también el nivel de satisfacción que genera entre lectores activos. Al aplicar el filtro de mínimo 50 calificaciones, se garantiza que el dato refleje una percepción colectiva y confiable, lo que lo convierte en un referente sólido para recomendaciones dentro de la aplicación.

_____
## ✳️6. Promedio de reseñas escritas por usuarios(con mas de 50)

In [70]:
# Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros
promedio_reseñas_texto = """
SELECT AVG(total_reviews) AS promedio_reseñas
FROM (
    select username, count(*) as total_reviews
    FROM reviews
    where username in (
        select username
        from ratings
        group BY username
        having count(*) > 50
    )
    group by username
) as subquery;
"""

df_promedio_reseñas_texto = pd.io.sql.read_sql(promedio_reseñas_texto, con=engine)

# Mostramos las primeras filas del dataframe
print(f'El promedio de reseñas escritas por usuarios con mas de 50 calificaciones es de:\n\t{df_promedio_reseñas_texto.iloc[0,0]:.2f} reseñas por usuario.')

El promedio de reseñas escritas por usuarios con mas de 50 calificaciones es de:
	24.33 reseñas por usuario.


## 🧫Resultados
Los usuarios más activos escribieron en promedio 24.33 reseñas de texto, lo que revela un segmento comprometido que podría ser clave para estrategias de fidelización y recomendación.

_____


# 🧾 Conclusión general
El análisis permitió extraer insights clave sobre publicaciones, comportamiento de usuarios y calidad percibida de los libros. Las consultas SQL aplicadas ofrecieron trazabilidad y claridad, reforzando la utilidad de esta base de datos para diseñar productos centrados en lectores activos y contenido de alto valor.
____